In [1]:
import seaborn as sns
import pandas as pd
import json
import matplotlib.pyplot as plt
import altair as alt
import numpy as np
import collections
import os
import more_itertools as mit
from Bio.Seq import Seq

In [2]:
sexChromosomes = ['CP068255.2','NC_060947.1','CP086569.2','NC_060948.1',
                 'CM055469.2','CM055470.2','NC_073248.2','NC_011120.1','CM054605.2','CM054581.2',
                 'CM054457.2','NC_072421.2','CM054458.2','NC_072422.2',
                  'CM055495.2','NC_073272.2','CM055496.2','NC_073273.2','CM054483.2','CM054507.2',
                  'CM054702.2','NC_072008.2','CM054703.2','NC_072009.2',
                  'CM054653.2','NC_072396.2','CM054654.2','NC_072397.2'
                 ]

In [3]:
chromAlias = pd.read_csv("/home/mark/Desktop/TE_Leaves/ChromosomeNames.alias.txt",sep='\t')

In [4]:
speciesSwitch={'Jim_GGO':'GorGor', 'PR00251_PPA':'PanPan'}
speciesSwitch2={y:x for x,y in speciesSwitch.items()}

speciesDict={y:{} for x,y in speciesSwitch.items()}

for species in speciesDict.keys():
    print(species)
    testDF = chromAlias[chromAlias['Sample']==str(speciesSwitch2[species])].copy()
    testDict=collections.Counter(testDF['Orig'])
    
    for row in testDF.index:
        if testDict[testDF.at[row,'Orig']]==2:
            if 'NC_' in str(testDF.at[row,'RefSeq']):
                speciesDict[species][testDF.at[row,'Genbank']]=testDF.at[row,'Orig'].split("_")[1]
            else:
                continue
        else:
            speciesDict[species][testDF.at[row,'Genbank']]=testDF.at[row,'Orig'].split("_")[1]

GorGor
PanPan


In [50]:
speciesDict

{'GorGor': {'CM054582.2': 'mat',
  'CM055446.2': 'pat',
  'CM055456.2': 'mat',
  'CM054568.2': 'pat',
  'CM055457.2': 'mat',
  'CM054569.2': 'pat',
  'CM054594.2': 'mat',
  'CM055458.2': 'pat',
  'CM054595.2': 'mat',
  'CM055459.2': 'pat',
  'CM054596.2': 'mat',
  'CM055460.2': 'pat',
  'CM054597.2': 'mat',
  'CM055461.2': 'pat',
  'CM054598.2': 'mat',
  'JAQQLJ020000074.1': 'mat',
  'JAQQLJ020000075.1': 'mat',
  'JAQQLJ020000076.1': 'mat',
  'JAQQLJ020000077.1': 'mat',
  'JAQQLJ020000078.1': 'mat',
  'JAQQLJ020000079.1': 'mat',
  'JAQQLJ020000080.1': 'mat',
  'JAQQLJ020000081.1': 'mat',
  'JAQQLJ020000082.1': 'mat',
  'JAQQLJ020000083.1': 'mat',
  'JAQQLJ020000084.1': 'mat',
  'JAQQLJ020000085.1': 'mat',
  'JAQQLJ020000086.1': 'mat',
  'JAQQLJ020000087.1': 'mat',
  'JAQQLJ020000088.1': 'mat',
  'JAQQLJ020000089.1': 'mat',
  'JAQQLJ020000090.1': 'mat',
  'JAQQLJ020000091.1': 'mat',
  'JAQQLJ020000092.1': 'mat',
  'JAQQLJ020000093.1': 'mat',
  'JAQQLJ020000094.1': 'mat',
  'JAQQLJ020000

In [8]:
from tqdm import tqdm
filteredDict={}
directory='/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Part3_DuplicationFiltered/'
outDirectory='/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Part4_FilteredandFixed/'

for csvFile in os.listdir(directory):
    if '_Final' in str(csvFile):
        print(csvFile)
        species = str(csvFile.split("_")[0])
        df1 = pd.read_csv(directory+csvFile).set_index("ID")
        df1['CHROM']=[x.split(":")[0] for x in df1.index]
        goodRows=[]
        for row in df1.index:
            if str(df1.at[row,'TE_Designation'])=='SINE/Alu' and float(df1.at[row,'TE_Percentage'])>=.70:
                goodRows.append(row)
            elif str(df1.at[row,'TE_Designation'])=='LINE/L1' and float(df1.at[row,'TE_Percentage'])>=.20:
                goodRows.append(row)

            elif str(df1.at[row,'TE_Designation'])=='Retroposon/SVA' and float(df1.at[row,'TE_Percentage'])>=.20:
                goodRows.append(row)

            else:
                continue

        df = df1.loc[goodRows].copy()
        
        df['Fixed']='NO'
        if species == 'hs1':
            pass
            
        elif species == 'GorGor' or species == 'PanPan':
            
            for row in df.index:
                
                parentContig = speciesDict[species][row.split(":")[0]]
                if parentContig == 'mat':
                    matchContig ='pat'
                else:
                    matchContig ='mat'
                    
                    
                if str(df.at[row,'All_Matches']) == str(species)+"-"+str(matchContig):
                    if float(df.at[row,str(species)+"-"+str(matchContig)+"_BestHitValue"])<=.2:
                        df.at[row,'Fixed']='Yes'
                    else:
                        continue
                else:
                    continue
            
                
        else:
            for row in df.index:
                if str(df.at[row,'All_Matches']) == str(species)+"-alt" :
                    if float(df.at[row,species+"-alt_BestHitValue"])<=.2:
                        df.at[row,'Fixed']='Yes'
                    else:
                        continue
                else:
                    continue
        
        print(collections.Counter(df['Fixed']))
        df2 = df[df['Fixed']=='Yes'].copy()
        print(collections.Counter(df2['TE_Designation']))
        df.to_csv(outDirectory+species+"_FinalTEPercentageFiltered_FixedAnnotation_06-26-2024.csv")
    else:
        continue

PanTro_MergedDup_Clean_Final_06-26-2024.csv
Counter({'Yes': 2997, 'NO': 889})
Counter({'SINE/Alu': 1641, 'LINE/L1': 922, 'Retroposon/SVA': 434})
GorGor_MergedDup_Clean_Final_06-26-2024.csv
Counter({'Yes': 8971, 'NO': 2852})
Counter({'SINE/Alu': 4926, 'LINE/L1': 2667, 'Retroposon/SVA': 1378})
PanPan_MergedDup_Clean_Final_06-26-2024.csv
Counter({'Yes': 2984, 'NO': 1026})
Counter({'SINE/Alu': 1747, 'LINE/L1': 804, 'Retroposon/SVA': 433})
hs1_MergedDup_Clean_Final_06-26-2024.csv
Counter({'NO': 12697})
Counter()
PonAbe_MergedDup_Clean_Final_06-26-2024.csv
Counter({'NO': 2614, 'Yes': 1090})
Counter({'LINE/L1': 942, 'Retroposon/SVA': 113, 'SINE/Alu': 35})
PonPyg_MergedDup_Clean_Final_06-26-2024.csv
Counter({'NO': 2355, 'Yes': 1569})
Counter({'LINE/L1': 1325, 'Retroposon/SVA': 190, 'SINE/Alu': 54})


In [9]:
speciesDict['GorGor']

{'CM054582.2': 'mat',
 'CM055446.2': 'pat',
 'CM055456.2': 'mat',
 'CM054568.2': 'pat',
 'CM055457.2': 'mat',
 'CM054569.2': 'pat',
 'CM054594.2': 'mat',
 'CM055458.2': 'pat',
 'CM054595.2': 'mat',
 'CM055459.2': 'pat',
 'CM054596.2': 'mat',
 'CM055460.2': 'pat',
 'CM054597.2': 'mat',
 'CM055461.2': 'pat',
 'CM054598.2': 'mat',
 'JAQQLJ020000074.1': 'mat',
 'JAQQLJ020000075.1': 'mat',
 'JAQQLJ020000076.1': 'mat',
 'JAQQLJ020000077.1': 'mat',
 'JAQQLJ020000078.1': 'mat',
 'JAQQLJ020000079.1': 'mat',
 'JAQQLJ020000080.1': 'mat',
 'JAQQLJ020000081.1': 'mat',
 'JAQQLJ020000082.1': 'mat',
 'JAQQLJ020000083.1': 'mat',
 'JAQQLJ020000084.1': 'mat',
 'JAQQLJ020000085.1': 'mat',
 'JAQQLJ020000086.1': 'mat',
 'JAQQLJ020000087.1': 'mat',
 'JAQQLJ020000088.1': 'mat',
 'JAQQLJ020000089.1': 'mat',
 'JAQQLJ020000090.1': 'mat',
 'JAQQLJ020000091.1': 'mat',
 'JAQQLJ020000092.1': 'mat',
 'JAQQLJ020000093.1': 'mat',
 'JAQQLJ020000094.1': 'mat',
 'JAQQLJ020000095.1': 'mat',
 'JAQQLJ020000096.1': 'mat',
 'J